In [3]:
# FASE 4_PROYECTO ZTNA_202016907_Proyecto de Grado_ANDRES_YORMAN_AIDA

# CASO DE USO #2: Análisis de postura de seguridad con Tenable.io

# Requisitos: pip install pytenable pandas matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tenable.io import TenableIO
from datetime import datetime, timedelta
import json
import os
from google.colab import files

# Configuración de la API de Tenable.io
# En un entorno de producción, usariamos variables de entorno o un sistema de gestión de secretos
ACCESS_KEY = "your_tenable_access_key"  # Se usa una clave de acceso
SECRET_KEY = "your_tenable_secret_key"  # Se usa una clave secreta

# Inicializar cliente de Tenable.io
def initialize_tenable_client():
    try:
        client = TenableIO(ACCESS_KEY, SECRET_KEY)
        return client
    except Exception as e:
        print(f"Error al inicializar el cliente de Tenable.io: {e}")
        return None

# Obtener vulnerabilidades de alta severidad
def get_high_severity_vulnerabilities(client, days_back=30):
    try:
        # Definir el rango de fechas
        end_date = datetime.utcnow()
        start_date = end_date - timedelta(days=days_back)

        # Filtrar por vulnerabilidades críticas y altas
        vulns = client.exports.vulns(
            severity=['critical', 'high'],
            last_found=start_date,
            num_assets=500  # Limitar a 500 activos para este ejemplo pero puede ser un numero mayor
        )

        vuln_list = []
        for vuln in vulns:
            vuln_list.append(vuln)

        print(f"Se encontraron {len(vuln_list)} vulnerabilidades críticas/altas")
        return vuln_list
    except Exception as e:
        print(f"Error al obtener vulnerabilidades: {e}")
        return []

# Procesar y analizar vulnerabilidades
def analyze_vulnerabilities(vulnerabilities):
    if not vulnerabilities:
        return None, None

    # Convertir a DataFrame
    df = pd.DataFrame(vulnerabilities)

    # Asegurarse de que todas las columnas necesarias existen
    required_columns = ['plugin_id', 'severity', 'plugin_name', 'asset_uuid',
                         'asset_ip', 'asset_hostname', 'first_found', 'last_found',
                         'vpr_score', 'cvss_v3_score']

    for col in required_columns:
        if col not in df.columns:
            df[col] = None

    # Limpiar y preparar datos
    # Convertir fechas a formato datetime
    for date_col in ['first_found', 'last_found']:
        if date_col in df.columns:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

    # Calcular tiempo de exposición en días
    if 'first_found' in df.columns and 'last_found' in df.columns:
        df['exposure_days'] = (df['last_found'] - df['first_found']).dt.days

    # Asignar puntaje de prioridad basado en severidad, CVSS y VPR
    df['priority_score'] = 0

    # Priorizar por severidad
    if 'severity' in df.columns:
        severity_map = {'critical': 4, 'high': 3, 'medium': 2, 'low': 1, 'info': 0}
        df['severity_numeric'] = df['severity'].map(lambda x: severity_map.get(x, 0))
        df['priority_score'] += df['severity_numeric'] * 25  # 25% del peso

    # Priorizar por CVSS v3
    if 'cvss_v3_score' in df.columns:
        df['cvss_v3_score'] = pd.to_numeric(df['cvss_v3_score'], errors='coerce').fillna(0)
        df['priority_score'] += df['cvss_v3_score'] * 25  # 25% del peso

    # Priorizar por VPR (Vulnerability Priority Rating)
    if 'vpr_score' in df.columns:
        df['vpr_score'] = pd.to_numeric(df['vpr_score'], errors='coerce').fillna(0)
        df['priority_score'] += df['vpr_score'] * 30  # 30% del peso

    # Priorizar por tiempo de exposición
    if 'exposure_days' in df.columns:
        # Normalizar días de exposición a una escala de 0-10
        max_days = df['exposure_days'].max()
        if max_days > 0:
            df['exposure_normalized'] = df['exposure_days'] / max_days * 10
            df['priority_score'] += df['exposure_normalized'] * 20  # 20% del peso

    # Ordenar por puntaje de prioridad
    df = df.sort_values('priority_score', ascending=False)

    # Agrupar vulnerabilidades por plugin para análisis
    vuln_by_plugin = df.groupby('plugin_name').agg({
        'plugin_id': 'first',
        'severity': 'first',
        'priority_score': 'mean',
        'asset_uuid': 'count'  # Contar activos afectados
    }).rename(columns={'asset_uuid': 'affected_assets'})

    vuln_by_plugin = vuln_by_plugin.sort_values('affected_assets', ascending=False)

    return df, vuln_by_plugin

# Generar visualizaciones
def generate_visualizations(df, vuln_by_plugin):
    if df is None or vuln_by_plugin is None:
        return

    # Crear directorio para visualizaciones
    os.makedirs('tenable_analysis', exist_ok=True)

    # 1. Distribución de vulnerabilidades por severidad
    plt.figure(figsize=(10, 6))
    severity_counts = df['severity'].value_counts()
    ax = severity_counts.plot(kind='bar', color=['darkred', 'red', 'orange', 'yellow', 'blue'])
    plt.title('Distribución de Vulnerabilidades por Severidad')
    plt.xlabel('Severidad')
    plt.ylabel('Número de Vulnerabilidades')
    plt.tight_layout()
    plt.savefig('tenable_analysis/severity_distribution.png')
    plt.close()

    # 2. Top 10 vulnerabilidades por número de activos afectados
    plt.figure(figsize=(12, 8))
    top_vulns = vuln_by_plugin.head(10)
    ax = top_vulns['affected_assets'].plot(kind='barh', color='lightcoral')
    plt.title('Top 10 Vulnerabilidades por Activos Afectados')
    plt.xlabel('Número de Activos')
    plt.ylabel('Vulnerabilidad')
    ax.set_yticklabels(top_vulns.index)
    plt.tight_layout()
    plt.savefig('tenable_analysis/top_vulnerabilities.png')
    plt.close()

    # 3. Mapa de calor de activos y vulnerabilidades
    if len(df) > 0 and 'asset_hostname' in df.columns and 'plugin_name' in df.columns:
        # Tomar una muestra manejable para el mapa de calor
        pivot_data = df.groupby(['asset_hostname', 'severity']).size().unstack().fillna(0)

        if not pivot_data.empty and pivot_data.shape[0] <= 20:  # Limitar a 20 hosts para legibilidad
            plt.figure(figsize=(12, 10))
            sns.heatmap(pivot_data, cmap='YlOrRd', annot=True, fmt='g')
            plt.title('Mapa de Calor de Vulnerabilidades por Host')
            plt.tight_layout()
            plt.savefig('tenable_analysis/vulnerability_heatmap.png')
            plt.close()

    # 4. Correlación entre tiempo de exposición y severidad
    if 'exposure_days' in df.columns and 'severity_numeric' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='severity', y='exposure_days', data=df, palette='YlOrRd')
        plt.title('Tiempo de Exposición por Severidad')
        plt.xlabel('Severidad')
        plt.ylabel('Días de Exposición')
        plt.tight_layout()
        plt.savefig('tenable_analysis/exposure_by_severity.png')
        plt.close()

# Generar reporte de remediación
def generate_remediation_report(df, vuln_by_plugin):
    if df is None or vuln_by_plugin is None:
        return

    # Crear un DataFrame para el reporte de remediación
    top_vulns = vuln_by_plugin.head(20).reset_index()  # Top 20 vulnerabilidades

    # Obtener detalles adicionales para cada vulnerabilidad
    remediation_report = []

    for _, row in top_vulns.iterrows():
        plugin_id = row['plugin_id']
        plugin_name = row['plugin_name']

        # Filtrar vulnerabilidades por plugin_id
        plugin_vulns = df[df['plugin_id'] == plugin_id]

        # Obtener hosts afectados
        affected_hosts = plugin_vulns['asset_hostname'].unique().tolist()
        affected_ips = plugin_vulns['asset_ip'].unique().tolist()

        # Limitar la lista de hosts para el informe
        if len(affected_hosts) > 5:
            host_display = ', '.join(affected_hosts[:5]) + f' y {len(affected_hosts)-5} más'
        else:
            host_display = ', '.join([h for h in affected_hosts if h])

        remediation_report.append({
            'Vulnerability': plugin_name,
            'Severity': row['severity'],
            'Priority Score': round(row['priority_score'], 2),
            'Affected Assets': row['affected_assets'],
            'Sample Hosts': host_display
        })

    # Crear DataFrame de reporte
    report_df = pd.DataFrame(remediation_report)

    # Guardar reporte en CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_file = f'tenable_analysis/remediation_report_{timestamp}.csv'
    report_df.to_csv(report_file, index=False)

    # También guardar como HTML para mejor visualización
    html_file = f'tenable_analysis/remediation_report_{timestamp}.html'
    report_df.to_html(html_file, index=False)

    return report_file, html_file

# Función principal
def main():
    print("Iniciando análisis de postura de seguridad con Tenable.io...")

    # 1. Inicializar cliente de Tenable
    client = initialize_tenable_client()
    if not client:
        return

    # 2. Obtener vulnerabilidades de alta severidad
    vulnerabilities = get_high_severity_vulnerabilities(client)

    # 3. Analizar vulnerabilidades
    df, vuln_by_plugin = analyze_vulnerabilities(vulnerabilities)

    if df is None:
        print("No hay datos suficientes para el análisis.")
        return

    # 4. Generar visualizaciones
    generate_visualizations(df, vuln_by_plugin)

    # 5. Generar reporte de remediación
    report_file, html_file = generate_remediation_report(df, vuln_by_plugin)

    print(f"\nAnálisis completo. Se procesaron {len(df)} vulnerabilidades.")
    print(f"Reporte de remediación guardado en: {report_file}")

    # 6. Si se ejecuta en Colab, descargar los archivos generados
    try:
        from google.colab import files
        files.download(report_file)
        files.download(html_file)

        # Descargar también las visualizaciones
        for viz_file in os.listdir('tenable_analysis'):
            if viz_file.endswith('.png'):
                files.download(f'tenable_analysis/{viz_file}')
    except:
        print(f"Los archivos se han guardado en el directorio 'tenable_analysis'")

# Punto de entrada principal
if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'tenable'